### **1. Paramètres**

In [5]:
# Importation des bibliothèques
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from functions_extract import (jointure_parcelle, jointure_pixel, 
                       extract_confidence_values, extract_lidar_values)

In [6]:
# Définition des paramètres
annees = [2017, 2018, 2019, 2020, 2021, 2022]  # Liste des années à traiter
zones = ['T30TYP', 'T30TYQ', 'T31TCJ', 'T31TGL', 'T31UEP']  # Liste des zones (tuiles)
lidar_metrics = ['grid_CC', 'grid_ENL', 'grid_MOCH', 'grid_PAI', 'grid_VCI'] # métriques lidar

# Chemins des répertoires et fichiers
output_dir_confidence = '../data_final/raster/confidence'
output_dir_lidar = '../data_final/raster/lidar' 
gpkg_parcelles_parcelle = '../data_final/vector/peupleraies_lidar_parcelle.gpkg'
gpkg_parcelles_pixel = '../data_final/vector/peupleraies_lidar_pixel.gpkg'
output_dir_csv = '../data_final/tableaux'  # Répertoire pour sauvegarder les fichiers CSV

# Création du répertoire de sortie pour les tableaux CSV s'il n'existe pas
os.makedirs(output_dir_csv, exist_ok=True)

# Chargement des vecteurs à partir des fichiers GeoPackage
peupleraies_parcelle = gpd.read_file(gpkg_parcelles_parcelle, layer='peupleraies_merged_parcelle')
peupleraies_pixel = gpd.read_file(gpkg_parcelles_pixel, layer='peupleraies_merged_pixel')

 ### **2. Extraction Raster de confidence** 

In [7]:
df_parcelle_all = []

# Boucle sur chaque zone pour extraire les valeurs
for zone in zones:
    conf_raster = os.path.join(output_dir_confidence, f"confidence_masked_{zone}.tif")
    df_conf = extract_confidence_values(conf_raster, annees, nodata=0)

    if df_conf is not None and not df_conf.empty:
        # Arrondi des coordonnées pour assurer la précision
        df_conf['x'] = df_conf['x'].round(2)
        df_conf['y'] = df_conf['y'].round(2)

        # Jointure spatiale à l'échelle parcelle
        df_merge = jointure_parcelle(df_conf, peupleraies_parcelle)

        if not df_merge.empty:
            df_parcelle_all.append(df_merge)

# Regroupement et exportation des résultats finaux
if df_parcelle_all:
    df_parcelle_final = pd.concat(df_parcelle_all, ignore_index=True)
    df_parcelle_final.to_csv(os.path.join(output_dir_csv, 'df_parcelle.csv'), index=False)
    print("CSV final à l'échelle parcelle généré : df_parcelle.csv")

CSV final à l'échelle parcelle généré : df_parcelle.csv


 ### **3. Extraction Raster de confidence + Lidar** 

In [8]:
df_pixel_all = []

# Boucle sur chaque zone pour traiter les données
for zone in zones:
    # Extraction des valeurs de confiance
    conf_raster = os.path.join(output_dir_confidence, f"confidence_masked_{zone}.tif")
    df_conf = extract_confidence_values(conf_raster, annees, nodata=0)
    if df_conf is None or df_conf.empty:
        continue

    # Arrondi des coordonnées pour garantir la correspondance
    df_conf['x'] = df_conf['x'].round(2)
    df_conf['y'] = df_conf['y'].round(2)

    # Récupération des métriques LiDAR (sauf pour T31UEP)
    lidar_raster_paths = {}
    if zone != 'T31UEP':
        for metric in lidar_metrics:
            r_path = os.path.join(output_dir_lidar, f"{metric}_masked_{zone}.tif")
            lidar_raster_paths[metric] = r_path

    # Fusion des données de confiance et des métriques LiDAR
    if lidar_raster_paths:
        df_lidar = extract_lidar_values(lidar_raster_paths, nodata=-999)
        if not df_lidar.empty:
            df_lidar['x'] = df_lidar['x'].round(2)
            df_lidar['y'] = df_lidar['y'].round(2)

        # Fusion (left join) des données de confiance avec les métriques LiDAR
        df_pixel_merged = pd.merge(df_conf, df_lidar, on=['x', 'y'], how='left')
        df_pixel_merged = df_pixel_merged[df_pixel_merged['valeur'].notnull()]
    else:
        # Si aucune donnée LiDAR n'est disponible, ajoute des colonnes vides
        df_pixel_merged = df_conf.copy()
        for metric in lidar_metrics:
            df_pixel_merged[metric] = pd.NA

    # Jointure spatiale à l'échelle pixel
    df_final_pixel = jointure_pixel(df_pixel_merged, peupleraies_pixel)
    if not df_final_pixel.empty:
        df_pixel_all.append(df_final_pixel)

# Regroupement et exportation des résultats finaux
if df_pixel_all:
    df_pixel_final = pd.concat(df_pixel_all, ignore_index=True)
    df_pixel_final.to_csv(os.path.join(output_dir_csv, 'df_pixel.csv'), index=False)
    print("CSV final à l'échelle pixel généré : df_pixel.csv")

C:\Users\caslu\AppData\Local\Temp\ipykernel_34116\293416774.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pixel_final = pd.concat(df_pixel_all, ignore_index=True)


CSV final à l'échelle pixel généré : df_pixel.csv
